<img src="https://electronaut.info/wp-content/uploads/2017/05/Tubescape_jupyter2.jpg">

<h1><font color = b30000>TUBE DATA MATCHER for uTRACER FILES</font></h1><h5>Anode current for a sweep of grid voltages while anode voltage is constant</h5>


# STEP 1 — Import and process tube data, build dataframe.

In [2]:
from ElectronauTracer_Importer import *
df.head()


,anode_voltage,tube_type,tube_ID,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
0,325,5749,726,68.367,43.693,28.669,21.741,16.96,13.277,10.509,8.422,6.808,5.582,4.635,3.882,3.315
1,325,5749,691,58.363,34.306,20.707,15.06,11.242,8.397,6.257,4.69,3.561,2.798,2.283,1.916,1.649
2,325,5749,646,85.052,51.933,33.207,24.308,18.414,13.955,10.633,8.141,6.287,4.954,3.97,3.249,2.737
3,325,5749,652,79.473,50.38,33.431,24.885,19.133,14.799,11.539,9.087,7.228,5.838,4.771,3.958,3.367
4,325,5749,450,78.117,47.47,30.216,22.023,16.41,12.29,9.273,7.096,5.534,4.449,3.676,3.077,2.602



# STEP 2: Show the statistics of the tube data set

In [3]:
# For some reason, .describe is requiring .astype(float) in order to show the stats.
df_stats = df.astype(float).describe()
df_stats.loc[:, 'tube_type'] = df_stats.loc[:, 'tube_type'].astype(int)
df_stats.loc[:, 'tube_ID'] = df_stats.loc[:, 'tube_ID'].astype(int)
df_stats

,anode_voltage,tube_type,tube_ID,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
count,237.0,237,237,237.000000,237.000000,237.000000,237.000000,237.000000,237.000000,237.000000,237.000000,237.000000,237.000000,237.000000,237.000000,237.000000
mean,325.0,5749,590,73.857797,44.625152,28.400886,20.730118,15.564886,11.767468,8.965325,6.909629,5.424899,4.371637,3.615937,3.047371,2.610781
std,0.0,0,114,10.237086,7.023709,5.242167,4.132572,3.321102,2.713109,2.255561,1.896899,1.611584,1.381073,1.191157,1.033530,0.905884
min,325.0,5749,361,44.127000,25.577000,13.355000,8.573000,5.914000,4.121000,2.853000,1.962000,1.344000,0.945000,0.697000,0.540000,0.447000
25%,325.0,5749,506,65.858000,39.711000,24.776000,17.848000,13.226000,9.816000,7.298000,5.448000,4.202000,3.308000,2.690000,2.290000,1.909000
50%,325.0,5749,601,74.370000,45.008000,28.866000,21.171000,15.827000,11.829000,8.963000,6.927000,5.385000,4.371000,3.638000,3.077000,2.628000
75%,325.0,5749,687,81.949000,50.027000,32.563000,24.000000,18.024000,13.680000,10.633000,8.361000,6.633000,5.370000,4.495000,3.820000,3.279000
max,325.0,5749,791,96.955000,59.669000,38.891000,29.300000,23.169000,18.486000,14.874000,12.080000,9.897000,8.171000,6.782000,5.665000,4.826000


# STEP 3: Plot a graph of all tubes


In [4]:
# Plot a line graph of all tubes

from bokeh.plotting import figure, output_file, show

output_file("Tube_Batch_Statistics.html")

# Configure the size, title, etc.
p = figure(plot_width=1000, plot_height=750, title="Plate Current as a function of Grid Voltage")
p.title.text_color = "black"

# Make a list for the x_values by chopping off all the "bias_" column name prefixes and converting to ints
temp_columns = list(df_stats.columns)
stats_columns = []
x_values = [int(i[5:]) for i in temp_columns if i.startswith('Bias_') == True]        

# Add all rows of tubes
for i in range(len(df.index)):
    row = df.iloc[i]
    row = list(row)
    row = row[3:]
    p.line(x_values, row, line_width=2)

# Set axis labels
p.xaxis.axis_label = "Grid Voltage (V)"
p.xaxis.axis_label_text_color = "#aa6666"
p.yaxis.axis_label = "Plate Current (mA)"

# Set grid lines
p.ygrid.minor_grid_line_color = 'navy'
p.ygrid.minor_grid_line_alpha = 0.2
p.xgrid.minor_grid_line_color = 'navy'
p.xgrid.minor_grid_line_alpha = 0.2

# Set background color
p.background_fill_color = "beige"
p.background_fill_alpha = 0.5

# Show the plot
show(p)

# STEP 5: Calculate the differences between all tubes

In [5]:
from ipywidgets import FloatProgress
from IPython.display import display
import time

start_time = time.time() 

def build_difference_df(df, df_stats):
    
    # Display a progress bar
    build_diff_progress_bar = FloatProgress(min=0, max=100)
    display(build_diff_progress_bar)
       
    # Make a list of all the columns in the dataframe
    cols = list(df.columns)
    
    # Make a list of just the bias columns (*** Should re-write this as a list comprehension)    
#     bias_list2 = [bias_list2.append(i) for i in cols if i.startswith('Bias_') == True]

    bias_list = []
    for i in cols:
        if i.startswith('Bias_') == True:
            bias_list.append(i)
    
    
    # Make an empty list to hold the calculated values, to be used to build the difference dataframe
    diff_squared_list = []
    
    # Iterate through the index of tubes in the master dataframe to select a reference tube
    for each_ref_tube in df.index:
        
        # Get the next reference tube data, as a series
        ref_tube = df.iloc[each_ref_tube]
        ref_tube_ID = ref_tube.tube_ID

        # Iterate through the index of tubes in the master dataframe to select a match tube
        for each_match_tube in df.index:
            
            # Get the next tube data to be compared, as a series
            match_tube = df.iloc[each_match_tube]
            match_tube_ID = match_tube.tube_ID
            
            # Dictionary container to hold the difference variables for each tube
            temp_dict = {}
            
            # Compute the squares of the differences in each current measurement
            error_sum = 0
            for bias in enumerate(bias_list):
                mismatch = ((float(match_tube.loc[bias[1]]) - float(ref_tube.loc[bias[1]]))**2)
                temp_dict[bias[1]] = mismatch
                error_sum = error_sum + mismatch

            # Add other relevant key/value pairs to the dictionary  
            temp_dict['ref_tube_ID'] = ref_tube_ID
            temp_dict['match_tube_ID'] = match_tube_ID
            temp_dict['error_sum'] = error_sum

            # Append the tube's calculated values to the diff_squared_list
            diff_squared_list.append(temp_dict)
            
            # Update the progress bar
            build_diff_progress_bar.value += 1

    
    # Make an ordered list for the columns of the new difference dataframe
    new_cols_to_add = ['ref_tube_ID', 'match_tube_ID', 'error_sum']
    df_dif_cols = new_cols_to_add + bias_list
    df_dif = pd.DataFrame(diff_squared_list, columns = df_dif_cols)

    return df_dif
    
    
df_dif = build_difference_df(df, df_stats)

# Produce a brief report:
total_rows = len(df_dif)
total_columns = len(df_dif.columns)
total_datapoints = total_rows * total_columns
stop_time = time.time()
elapsed_time = stop_time - start_time
print("It took", round(elapsed_time, 2), "seconds to calculate", 
      total_datapoints, "values, resulting in", len(df_dif), "rows.")

# Display the head of the difference dataframe
df_dif.head()

FloatProgress(value=0.0)

It took 25.14 seconds to calculate 898704 values, resulting in 56169 rows.


,ref_tube_ID,match_tube_ID,error_sum,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
0,726,726,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,726,691,415.208523,100.080016,88.115769,63.393444,44.635761,32.695524,23.814400,18.079504,13.927824,10.543009,7.750656,5.531904,3.865156,2.775556
2,726,646,377.980718,278.389225,67.897600,20.593444,6.589489,2.114116,0.459684,0.015376,0.078961,0.271441,0.394384,0.442225,0.400689,0.334084
3,726,652,209.431035,123.343236,44.715969,22.676644,9.884736,4.721929,2.316484,1.060900,0.442225,0.176400,0.065536,0.018496,0.005776,0.002704
4,726,450,121.346443,95.062500,14.265729,2.393209,0.079524,0.302500,0.974169,1.527696,1.758276,1.623076,1.283689,0.919681,0.648025,0.508369


# STEP 6: Find the best matches for a particular tube_ID

In [ ]:
# This function requires the tube_to_match and tube_set_size variable to be set manually, below.

def find_best_set_for_chosen_tube_ID(tube_to_match, tube_set_size):
    tube_to_match = str(tube_to_match)
    
    # Create a new dataframe showing only tube_set_size number of nearest matches to the chosen tube_to_match
    df_tube_set = df_dif[(df_dif.ref_tube_ID == tube_to_match)].sort_values('error_sum').head(tube_set_size)
    
    return df_tube_set

tube_to_match = 460
tube_set_size = 2
df_tube_set = find_best_set_for_chosen_tube_ID(tube_to_match, tube_set_size)
df_tube_set.head(tube_set_size)


# STEP 7:  Build a dataframe of the best matched sets in order

In [50]:
# math lib used for generating ordinal strings; i.e. ['1st', '2nd', '3rd'] etc.
import math

# Make a local version of df_dif that can be butchered
df_dif_copy = df_dif.copy()

def ask_tube_set_size():
    
    # Ask the user to enter the set size
    tube_set_size = input("Please enter the number of tubes per matched set:")
    
    # Build a list of tube names for the tube set size
    tube_sets_columns = ['Tube_' + str(1 + i) for i in range(int(tube_set_size))]    
    
    return tube_set_size, tube_sets_columns 


def find_best_set_for_chosen_tube_ID(df_dif_copy, tube_to_match, tube_set_size, tube_sets_columns):
    tube_to_match = str(tube_to_match)
        
    # Create a new dataframe showing only tube_set_size number of nearest matches to the chosen tube_to_match
    df_tube_set = df_dif_copy[(df_dif_copy.ref_tube_ID == tube_to_match)].sort_values('error_sum').head(int(tube_set_size))
        
    # Create the dictionary of the tubes in the set and the total score of the set
    tube_set_list = list(df_tube_set.match_tube_ID)
    tube_set_dict = dict(zip(tube_sets_columns, tube_set_list))   
    tube_set_score = df_tube_set.error_sum.sum()
    tube_set_dict.update({'tube_set_score':tube_set_score})

    return tube_set_dict


def find_best_set(df_dif_copy, tube_set_size, tube_sets_columns):
    
    # Make an empty list to hold the tube sets and their scores
    tube_sets_and_scores_list = []
    temp_dict = {}
    
    if 'tube_set_score' in tube_sets_columns:
        tube_sets_columns.remove('tube_set_score')

    for i in df_dif_copy.ref_tube_ID.unique():
        temp_dict = find_best_set_for_chosen_tube_ID(df_dif_copy, i, tube_set_size, tube_sets_columns)
        tube_sets_and_scores_list.append(temp_dict)

    # Add 'tube_set_score' column name at the front of the list
    if 'tube_set_score' not in tube_sets_columns: 
        tube_sets_columns.insert(0, 'tube_set_score')    
    
    # Create a dataframe for the tube sets called df_tube_sets 
    df_tube_sets = pd.DataFrame(tube_sets_and_scores_list, columns=tube_sets_columns)
    
    # Sort the tube_set_score column, take the top row (with the lowest score) as a new df 
    df_best_set_temp = df_tube_sets.sort_values('tube_set_score').head(1)
        
    return df_best_set_temp


# A function to plot a line graph of a set of tubes by passing in a list of tube numbers
def plot_tubes(list_of_tubes, df):
    
    output_file("Tube_Set_Statistics.html")

    # Configure the size, title, etc.
    p = figure(plot_width=1000, plot_height=750, title=str(list_of_tubes))
    p.title.text_color = "black"

    # Make a list for the x_values by chopping off all the "bias_" column name prefixes and converting to ints
    temp_columns = list(df.columns)
    x_values = [int(i[5:]) for i in temp_columns if i.startswith('Bias_') == True]        


    # Set axis labels
    p.xaxis.axis_label = "Grid Voltage (V)"
    p.xaxis.axis_label_text_color = "#aa6666"
    p.yaxis.axis_label = "Plate Current (mA)"

    # Set grid lines
    p.ygrid.minor_grid_line_color = 'navy'
    p.ygrid.minor_grid_line_alpha = 0.2
    p.xgrid.minor_grid_line_color = 'navy'
    p.xgrid.minor_grid_line_alpha = 0.2

    # Set background color
    p.background_fill_color = "beige"
    p.background_fill_alpha = 0.5

    for tube in list_of_tubes:
        row = ((df.loc[df['tube_ID'] == tube[0:3]]).values.tolist())[0]
        row = [float(i) for i in row[3:]]
        p.line(x_values, row, line_width=2)

    return p    


    
# Start a timer
start_time = time.time()   
    
# Make an empty dataframe to hold all the sets and their scores
df_best_tube_sets = pd.DataFrame()

# Get the tube_set_size and the tube_sets_columns from the ask_tube_set_size function
tube_set_size, tube_sets_columns = ask_tube_set_size()

# Get the best tube set from the batch
df_best_set_temp = find_best_set(df_dif_copy, tube_set_size, tube_sets_columns) 

# Display a progress bar
progress_bar_range = int(len(df)/int(tube_set_size))
best_matched_progress_bar = FloatProgress(min=0, max=progress_bar_range)
display(best_matched_progress_bar)

# Append the best tube set to the df_best_tube_sets dataframe
df_best_tube_sets = df_best_tube_sets.append(df_best_set_temp)

# Remove the best tube set from df_dif_copy
tube_delete_list = list(df_best_set_temp.iloc[0])
tube_delete_list.pop(0)
df_dif_copy = df_dif_copy[~df_dif_copy.ref_tube_ID.isin(tube_delete_list)]
df_dif_copy = df_dif_copy[~df_dif_copy.match_tube_ID.isin(tube_delete_list)]

# A function to generate ordinal strings; i.e. ['1st', '2nd', '3rd'] etc.
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(math.floor(n/10)%10!=1)*(n%10<4)*n%10::4])
ordinal_list = [ordinal(n) for n in range(1, int(len(df)))]

set_count = 1
print()
print("*** BEST! ***")
print(ordinal_list[0],"best tube set SCORE:", str(df_best_set_temp.tube_set_score.iloc[0]))
print("TUBES:", tube_delete_list)

# Update the progress bar
best_matched_progress_bar.value += 1

tube_matches_master_plot_list = []

while len(df_dif_copy.ref_tube_ID.unique()) >= int(tube_set_size):

    # Get all the rest of the sets from the batch
    df_best_set_temp = find_best_set(df_dif_copy, tube_set_size, tube_sets_columns)
    
    # Append the best tube set to the df_best_tube_sets dataframe
    df_best_tube_sets = df_best_tube_sets.append(df_best_set_temp)

    # Remove the best tube set from df_dif_copy
    tube_delete_list = list(df_best_set_temp.iloc[0])
    tube_delete_list.pop(0)

    df_dif_copy = df_dif_copy[~df_dif_copy.ref_tube_ID.isin(tube_delete_list)]
    df_dif_copy = df_dif_copy[~df_dif_copy.match_tube_ID.isin(tube_delete_list)]
    print()
    print(ordinal_list[set_count],"best tube set SCORE:", str(df_best_set_temp.tube_set_score.iloc[0]))
    print("TUBES:", tube_delete_list)
    tube_delete_list.insert(0, ordinal)

    tube_matches_master_plot_list.insert(1 ,tube_delete_list[1:])

    set_count += 1
    
    # Update the progress bar
    best_matched_progress_bar.value += 1

    
    



stop_time = time.time()
elapsed_time = stop_time - start_time
print()
print("It took", round(elapsed_time, 2), "seconds to produce", set_count, "tube sets.")

# print(tube_matches_master_plot_list[0])

# plot_tubes((list(df_best_tube_sets.iloc[0])[1:]), df)
    
# Build the plot object
# tube_plot = plot_tubes(tube_delete_list, df)

# Show the plot
# show(tube_plot)



best_set_list = list(df_best_tube_sets.iloc[0])[1:]
worst_set = df_best_tube_sets.tail(1)
worst_set_list = list(worst_set.iloc[0])[1:]



Please enter the number of tubes per matched set:16


FloatProgress(value=0.0, max=14.0)


*** BEST! ***
1st best tube set SCORE: 38.79435300000003
TUBES: ['655', '597', '440', '663', '727', '654', '425', '601', '678', '416', '777', '738', '791', '775', '662', '717']

2nd best tube set SCORE: 45.43804299999998
TUBES: ['584', '622', '596', '575', '576', '590', '619', '599', '496', '585', '651', '567', '538', '549', '586', '572']

3rd best tube set SCORE: 46.14140000000006
TUBES: ['702', '472', '605', '706', '721', '594', '680', '550', '673', '595', '708', '666', '603', '729', '540', '405']

4th best tube set SCORE: 47.81206200000008
TUBES: ['690', '725', '684', '607', '454', '401', '361', '407', '737', '404', '661', '668', '448', '646', '628', '788']

5th best tube set SCORE: 55.28115600000004
TUBES: ['644', '547', '608', '534', '477', '548', '560', '669', '563', '491', '421', '639', '433', '683', '699', '362']

6th best tube set SCORE: 66.10696799999994
TUBES: ['552', '532', '724', '418', '611', '475', '609', '677', '716', '645', '688', '591', '410', '714', '718', '364']

7

In [51]:
print(tube_matches_master_plot_list)

    

[['584', '622', '596', '575', '576', '590', '619', '599', '496', '585', '651', '567', '538', '549', '586', '572'], ['378', '451', '665', '387', '679', '389', '635', '630', '606', '616', '570', '693', '386', '432', '719', '566'], ['376', '441', '743', '741', '399', '650', '370', '411', '772', '449', '652', '747', '720', '511', '728', '559'], ['653', '508', '667', '600', '587', '700', '507', '569', '473', '396', '413', '692', '670', '582', '726', '647'], ['439', '495', '697', '615', '629', '499', '369', '497', '561', '671', '427', '656', '604', '610', '492', '614'], ['691', '506', '589', '696', '593', '742', '711', '535', '502', '490', '542', '687', '703', '541', '602', '707'], ['701', '704', '767', '715', '627', '565', '568', '592', '633', '695', '555', '556', '723', '414', '789', '450'], ['641', '779', '748', '453', '443', '537', '660', '377', '373', '385', '379', '613', '766', '698', '403', '730'], ['480', '571', '456', '722', '681', '583', '544', '581', '709', '694', '562', '642', '6

In [52]:
print(worst_set)

    tube_set_score Tube_1 Tube_2 Tube_3 Tube_4 Tube_5 Tube_6 Tube_7 Tube_8  \
11     1738.514809    378    451    665    387    679    389    635    630   

   Tube_9 Tube_10 Tube_11 Tube_12 Tube_13 Tube_14 Tube_15 Tube_16  
11    606     616     570     693     386     432     719     566  


In [18]:
# PLOT LIST
# A function to plot a line graph of a set of tubes by passing in a list of tube numbers


def plot_tubes(list_of_tubes, df):
    
    output_file("Tube_Set_Statistics.html")

    # Configure the size, title, etc.
    p = figure(plot_width=1000, plot_height=750, title="Plate Current as a function of Grid Voltage")
    p.title.text_color = "black"

    # Make a list for the x_values by chopping off all the "bias_" column name prefixes and converting to ints
    temp_columns = list(df.columns)
    x_values = [int(i[5:]) for i in temp_columns if i.startswith('Bias_') == True]        


    # Set axis labels
    p.xaxis.axis_label = "Grid Voltage (V)"
    p.xaxis.axis_label_text_color = "#aa6666"
    p.yaxis.axis_label = "Plate Current (mA)"

    # Set grid lines
    p.ygrid.minor_grid_line_color = 'navy'
    p.ygrid.minor_grid_line_alpha = 0.2
    p.xgrid.minor_grid_line_color = 'navy'
    p.xgrid.minor_grid_line_alpha = 0.2

    # Set background color
    p.background_fill_color = "beige"
    p.background_fill_alpha = 0.5

    for tube in list_of_tubes:
        row = ((df.loc[df['tube_ID'] == tube[0:3]]).values.tolist())[0]
        row = [float(i) for i in row[3:]]
        p.line(x_values, row, line_width=2)

    return p    


# Build the plot object
tube_plot = plot_tubes(tubes_to_plot, df)

# Show the plot
show(tube_plot)

NameError: name 'tubes_to_plot' is not defined

# STEP 4: Show the distribution of a single bias voltage


In [ ]:
# Histogram of a single column

import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import numpy as np

sigma = df_stats.iat[2, 5]
mu = df_stats.iat[1, 5]
x = mu + sigma  * np.random.randn(437)
num_bins = 50

fig, ax = plt.subplots()

# the histogram of the data
n, bins, patches = ax.hist(x, num_bins, normed=1)

# add a 'best fit' line
y = mlab.normpdf(bins, mu, sigma)
ax.plot(bins, y, '--')
ax.set_xlabel('Current (mA)')
# ax.set_ylabel('Probability density')
ax.set_title(r'Histogram of -10V grid voltage')

# Tweak spacing to prevent clipping of ylabel
fig.tight_layout()
plt.show()

## Compute the total currents of every possible combination of quads

#### Prepare the data

In [ ]:
import itertools

# make a reference copy of the main df and set the index of the main df to be the tube_ID number
df_copy = df.copy()
df_copy.set_index('tube_ID', inplace=True)

# make a df containing only the best set
best_set_df = df_copy.loc[best_set_list]

# make a list of the bias voltages
bias_list = best_set_df.columns.tolist()[2:]

#### Make a list of all possible sets of 4

In [2]:
import itertools
# Make a list of all possible sets of 4 from the best_set list
combinations_list = []
for combination in itertools.combinations(best_set_list, 4):
    combinations_list.append(combination)


NameError: name 'best_set_list' is not defined

#### General function definitions

In [ ]:
# Function to extract the current from a specified tube and bias voltage
def get_current(tube, bias):
    
    # Get the current from the tube at the specified bias (returned as dataframe)
    buzzz = best_set_df.loc[[tube], [bias]]
    
    # Convert to dict (results in nested dict)
    buzz_dict = buzzz.to_dict()
    # Looks like:  {'Bias_x' : {tube_ID : current}}
    
    # Get the name of the first dict key
    buzz_key = str(list(buzz_dict.keys())[0])

    # Extract the nested dict
    buzz_value_dict = buzz_dict[buzz_key]

    # Get the name of the nested dict key
    buzz_value_key = str(list(buzz_value_dict.keys())[0])

    # extract the value and turn it into a float
    buzz = float(buzz_value_dict[buzz_value_key])
    
    return buzz

# Function to total up the currents for a given combo tuple and bias voltage
def add_currents(combo_tuple, bias):
    total_current = 0.0
    for eachtube in combo_tuple:
        zzz = get_current(eachtube, bias)
        total_current += zzz
    return {bias : total_current}

# Function to build a dictionary for each combination containing total currents from each bias voltage
def build_combination_data(eachcombo, bias_list):
    combo_big_dict = {}
    for eachbias in bias_list:
            bias_dict = add_currents(eachcombo, eachbias)
            combo_big_dict.update(bias_dict)
            temp_dict = {'Combination' : eachcombo}
            combo_big_dict.update(temp_dict)
    return combo_big_dict

# Function for building a new list including only valid complimentary quads
def trim_combo_list(combination_tuple, combinations_list):
    trimmed_combo_list = [i for i in combinations_list if (combination_tuple[0]) not in i]
    trimmed_combo_list = [i for i in trimmed_combo_list if (combination_tuple[1]) not in i]
    trimmed_combo_list = [i for i in trimmed_combo_list if (combination_tuple[2]) not in i]
    trimmed_combo_list = [i for i in trimmed_combo_list if (combination_tuple[3]) not in i]
    print(len(trimmed_combo_list), "valid complimentary quads.")
    return trimmed_combo_list


In [ ]:
# Hard-coding an example
combination_tuple = combinations_list[0]

trimmed_combinations_list = trim_combo_list(combination_tuple, combinations_list)

print(len(trimmed_combinations_list), "valid combinations")

# trimmed_combination_list

#### build a dataframe of combinations and the total currents for each bias position

In [ ]:
# Build a dataframe showing the summed current for every tube and bias combination
# Prepare the master list for the dataframe

# Stopwatch
start_combos_time = time.time()


combos_data_list = []
for eachcombo in trimmed_combinations_list:
    thing = build_combination_data(eachcombo, bias_list)
    combos_data_list.append(thing)

    # Make a list of column names for the dataframe    
after_for_loop = time.time()
combo_columns = ['Combination'] + bias_list

# Build the dataframe
combo_df = pd.DataFrame(combos_data_list, columns=combo_columns)
after_df_generation = time.time()
print("For-loop took", (after_for_loop - start_combos_time), 'seconds.')
print("Dataframe generation took", (after_df_generation - after_for_loop), 'seconds.')
print("Total elapsed time:", (after_df_generation-start_combos_time))
print(len(combo_df), 'total possible combinations')
combo_df.head()    

#### Make a temp copy of combo_df and include only rows from the trimmed_combinations_list 

In [ ]:
# Make a temp copy of combo_df and include only rows from the trimmed_combinations_list 
temp_df2 = combo_df
temp_df2[temp_df2['Combination'].isin(trimmed_combinations_list)]

In [ ]:
# Sandbox

def mask(df, key, value):
    return df[df[key] == value]

pd.DataFrame.mask = mask

thing = combo_df['Combination'][0]

# combo_df.mask('A', 1).mask('D', 6)
temp_df = combo_df.mask('Combination', thing)
temp_df

In [ ]:
# Single-value bias matcher.

# TODO: Get the target current.  What is being matched?



def single_value_matcher(combo_df, combination_tuple, bias):
    combo_df = combo_df.sort_values([bias])
    


In [ ]:
# Build a function to mask the df by combination_tuple

def mask(df, key, value):
    return df[df[key] == value]

pd.DataFrame.mask = mask
combination_tuple_b = trimmed_combinations_list[3]

# combo_df.mask('A', 1).mask('D', 6)
temp_df = combo_df.mask('Combination', combination_tuple_b)
temp_df

In [ ]:
# # Make a simple matcher to find the closest matched currents at a single bias voltage

# test_bias = bias_list[0]

# totalcombos = combo_df.index.tolist()
# testcombo = totalcombos[0]
    
# def find_closest_combo(combo_df, bias):
    
#     # Sort the dataframe by the chosen bias column
#     combo_df = combo_df.sort_values([bias])

#     # Pick the combo to match (in this case, a fixed sample instead of an iterated loop)
#     totalcombos = combo_df.index.tolist()
#     testcombo = totalcombos[0]
#     nextcombo = totalcombos[1]
#     previouscombo = totalcombos[-1]
    
#     # Get the reference value
#     ref_value = combo_df.loc[testcombo, bias]

#     # Get the previous and next values to compare
#     next_value = combo_df.loc[nextcombo, bias]
#     previous_value = combo_df.loc[previouscombo, bias]
    
#     # Figure out which one is closer
#     diff_next = abs(float(ref_value) - float(next_value))
#     diff_previous = abs(float(ref_value) - float(previous_value))
    
#     # Return the closer match
#     if diff_next < diff_previous:
#         matchcombo = combo_df.loc[nextcombo, 'Combination']
#     else:
#         matchcombo = combo_df.loc[previouscombo, 'Combination']
#     return matchcombo

# matchedcombo = find_closest_combo(combo_df, test_bias)
# refcombo = combo_df.loc[testcombo, 'Combination']

# print("Refcombo:", refcombo, "  Current at bias voltage:", (combo_df.loc[testcombo, test_bias]))
# print("Matchcombo:", matchedcombo, "  Current at bias voltage:", (combo_df.loc[matchedcombo, test_bias]))



# # test_df = find_closest_combo(combo_df, test_bias)
# # test_df

# STEP 8: Plot a line graph of the best set


In [ ]:
# Plot a line graph of the best set

from bokeh.plotting import figure, output_file, show

output_file("Best Matched Tube Set.html")

# Configure the size, title, etc.
p = figure(plot_width=1000, plot_height=750, title="Plate Current as a function of Grid Voltage")
p.title.text_color = "black"

# Make a list for the x_values by chopping off all the "bias_" column name prefixes and converting to ints
temp_columns = list(df.columns)
x_values = [int(i[5:]) for i in temp_columns if i.startswith('Bias_') == True]        

for i in range(len(best_set)):
    row = df[df['tube_ID'] == best_set[i]]
    y_values = row.values.tolist()[0][3:]
    p.line(x_values, y_values, line_width=2)

# Set axis labels
p.xaxis.axis_label = "Grid Voltage (V)"
p.xaxis.axis_label_text_color = "#aa6666"
p.yaxis.axis_label = "Plate Current (mA)"

# Set grid lines
p.ygrid.minor_grid_line_color = 'navy'
p.ygrid.minor_grid_line_alpha = 0.2
p.xgrid.minor_grid_line_color = 'navy'
p.xgrid.minor_grid_line_alpha = 0.2

# Set background color
p.background_fill_color = "beige"
p.background_fill_alpha = 0.5

# Show the plot
show(p)

# STEP 9: Plot a line graph of the worst set


In [ ]:
# Plot a line graph of the worst set

from bokeh.plotting import figure, output_file, show

output_file("Worst Matched Tube Set.html")

# Configure the size, title, etc.
p = figure(plot_width=1000, plot_height=750, title="Plate Current as a function of Grid Voltage")
p.title.text_color = "black"

# Make a list for the x_values by chopping off all the "bias_" column name prefixes and converting to ints
temp_columns = list(df.columns)
x_values = [int(i[5:]) for i in temp_columns if i.startswith('Bias_') == True]        

for i in range(len(worst_set)):
    row = df[df['tube_ID'] == worst_set[i]]
    y_values = row.values.tolist()[0][3:]
    p.line(x_values, y_values, line_width=2)

# Set axis labels
p.xaxis.axis_label = "Grid Voltage (V)"
p.xaxis.axis_label_text_color = "#aa6666"
p.yaxis.axis_label = "Plate Current (mA)"

# Set grid lines
p.ygrid.minor_grid_line_color = 'navy'
p.ygrid.minor_grid_line_alpha = 0.2
p.xgrid.minor_grid_line_color = 'navy'
p.xgrid.minor_grid_line_alpha = 0.2

# Set background color
p.background_fill_color = "beige"
p.background_fill_alpha = 0.5

# Show the plot
show(p)

### Everything below this line is experimental

In [ ]:
df_best_tube_sets.tube_set_score[:15].describe()


In [ ]:
import seaborn as sns
sns.set()

df_sns = df_best_tube_sets[['tube_set_score']][:15].copy()

sns.pairplot(df_sns, 'tube_set_score')
sns.plt.show()


In [ ]:
from bokeh.charts import Histogram, output_file, show

p = Histogram(df_best_tube_sets['tube_set_score'], title="Tube Set Scores")

output_file("histogram.html",)

show(p)

# Other Miscellaneous stuff

##### Apply a function to a column
df.Bias_2.max()

In [ ]:
% lsmagic

In [ ]:
%pprint



In [ ]:
# Applymap is useful for applying a function to all data
# For example, converting all data from floats to ints
# df.loc[2:7, 'Bias_2': 'Bias_50'].applymap(int)

In [ ]:
# List all available magics
% lsmagic

In [ ]:
# Display environmental variables
env_variables = %env
current_path = env_variables.get('PATH')
current_path

In [ ]:
mydict = % env
mydict.keys()

In [ ]:
% env

In [ ]:
# % time x = range(10000)
% timeit x = range(10000)
max(x)